# Predicting Annual Damage - Simple Model

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from math import factorial, sqrt

import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import rv_discrete

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.learning_curve import learning_curve
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import Imputer, label_binarize, LabelEncoder, OneHotEncoder,\
MinMaxScaler, scale
from sklearn.cluster import KMeans
from sklearn.externals.six import StringIO
from IPython.display import Image  
from scipy.optimize import minimize
from scipy.stats.distributions import poisson
import pydot

from collections import Counter, defaultdict
import random
import re
random.seed(4444)

/home/lucia/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Reading Data

In [2]:
subset_all = pd.read_pickle('./dfs/subset_all')

## Model features

features = ['INJURIES', 'DEATHS', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 
            'EVENT_CATEGORY', 'BEGIN_YEAR', 'DURATION', 'STATE']
df = subset_all[features]
feat_cat = ['EVENT_CATEGORY', 'STATE']
for item in feat_cat:
    df[item] = df[item].astype('category')

/home/lucia/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Checking for missing values

In [3]:
l = 1.0*len(df)
for name in df.columns.values:
    ln = len(df[df[name].isnull()])
    print name, ln/l

dfn = df.dropna()
print 'Percentage of data with damage property&crops info: ', 100.0*len(dfn)/len(df)
print dfn.dtypes

INJURIES 0.0
DEATHS 0.0
DAMAGE_PROPERTY 0.420281760126
DAMAGE_CROPS 0.522487584474
EVENT_CATEGORY 0.0
BEGIN_YEAR 0.0
DURATION 0.0
STATE 0.0
Percentage of data with damage property&crops info:  45.7573221454
INJURIES              int64
DEATHS                int64
DAMAGE_PROPERTY     float64
DAMAGE_CROPS        float64
EVENT_CATEGORY     category
BEGIN_YEAR            int64
DURATION            float64
STATE              category
dtype: object


## Splitting the dataset into train and test sets

In [4]:
dfn_trainvalid = dfn.loc[dfn.BEGIN_YEAR < 2013, :]
dfn_test = dfn.loc[dfn.BEGIN_YEAR >= 2013, :]
print 'Percentage of train data: ', 100.0*len(dfn_trainvalid)/len(dfn)

Percentage of train data:  72.4070419074


# Mean Stats - Bootstrap - By Event Category

In [5]:
def get_mean_stats(df, main_col):
    return df.groupby([main_col]).mean()

In [6]:
def get_CI(col, size=1000):
    n = len(col)
    mean_list = []
    for i in range(size):
        sample = col.iloc[np.random.randint(0, n, size=round(n*0.3/0.7))]
        mean_list.append(sample.mean())
    mean_list = np.array(sorted(mean_list))
    mean = np.mean(mean_list)
    return (mean, np.percentile(mean_list,2.5), np.percentile(mean_list,97.5))

In [7]:
def bootstrap_means(df, main_col, damage_features):
    predictions = defaultdict(dict)
    for category in df[main_col].unique():
        for feature in damage_features:
            data = df.loc[df[main_col] == category, feature]
            predictions[feature][category] = get_CI(data)
    return predictions

In [8]:
def bootstrap_accuracy(results, predictions):
    count = 0
    for category in results.index.values:
        for feature in results.columns.values:
            value = results.ix[category,feature]
            val_tuple = predictions[feature][category]
            if (val_tuple[1] <= float(value)) & (float(value) <= val_tuple[2]):
                count += 1
    dims = (results.shape[0])*(results.shape[1])
    return 1.0*count/dims

In [9]:
damage_features = ['INJURIES','DEATHS','DAMAGE_PROPERTY','DAMAGE_CROPS']
main_col = 'EVENT_CATEGORY'
target_cols = [main_col] + damage_features

In [10]:
predictions = bootstrap_means(dfn_trainvalid, main_col, damage_features)

/home/lucia/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [11]:
print predictions

defaultdict(<type 'dict'>, {'DAMAGE_PROPERTY': {'Winter Weather': (16409.293089884934, 9057.1959621061505, 32045.557215796129), 'Tornado': (1394011.5694426298, 727004.18906854768, 2452513.6943633566), 'Hurricane': (99885538.39662984, 43874543.566298351, 179897947.48825967), 'Tide': (127349.22784030419, 28348.531844106466, 277167.39096958167), 'Flood': (4218542.9562857104, 534757.17556598166, 15844815.069229176), 'Heat': (271989.84160483244, 122117.65558117036, 516283.10945729114), 'Storm': (146845.89754942316, 102753.91702977022, 200945.51088597966), 'Wind': (101640.85176973206, 55099.96991354927, 165336.8764382785)}, 'DAMAGE_CROPS': {'Winter Weather': (119331.6861215736, 60700.499698023974, 208951.27612657391), 'Tornado': (26768.970705190808, 15230.751181045398, 53298.625723458128), 'Hurricane': (15350691.950143648, 8947103.6475138124, 24151089.812430941), 'Tide': (7.6476045627376443, 0.0, 33.764258555133082), 'Flood': (186424.26955380329, 125524.98695813313, 274963.91186018672), 'Hea

In [18]:
results = get_mean_stats(dfn_trainvalid, main_col)[damage_features]
print results, '\n'
#print bootstrap_accuracy(results, predictions)

                INJURIES    DEATHS  DAMAGE_PROPERTY  DAMAGE_CROPS
EVENT_CATEGORY                                                   
Flood           0.179770  0.024013     4.291519e+06  1.863786e+05
Heat            0.282906  0.035717     2.745186e+05  4.932792e+05
Hurricane       2.186761  0.134752     1.008096e+08  1.530431e+07
Storm           0.035160  0.005986     1.478084e+05  2.600693e+04
Tide            0.095797  0.035842     1.271405e+05  7.233627e+00
Tornado         0.839780  0.076893     1.391808e+06  2.666468e+04
Wind            0.029840  0.005737     1.030949e+05  1.862461e+04
Winter Weather  0.068832  0.014665     1.664656e+04  1.206561e+05 



In [19]:
stormcounts = pd.read_csv("./dfs/avestormcounts.csv")
stormcounts = stormcounts.loc[stormcounts.EVENT_CATEGORY != 'Other',:]
stormcounts.head()

,STATE,EVENT_CATEGORY,COUNT
0,alabama,Flood,86.25
1,alabama,Heat,93.15
2,alabama,Hurricane,9.60
3,alabama,Storm,276.40
4,alabama,Tide,1.20


In [20]:
def ave_col(row, storm_dict):
    return row[2]*storm_dict[row[1]][0]

def CIlower_col(row, storm_dict):
    return row[2]*storm_dict[row[1]][1]

def CIupper_col(row, storm_dict):
    return row[2]*storm_dict[row[1]][2]

In [22]:
#for key in predictions:
#    stormcounts[key] = stormcounts.apply(func = ave_col, axis = 1, storm_dict = predictions[key])
#    stormcounts[key+"_LOWER"] = stormcounts.apply(func = CIlower_col, axis = 1, storm_dict = predictions[key])
#    stormcounts[key+"_UPPER"] = stormcounts.apply(func = CIupper_col, axis = 1, storm_dict = predictions[key])
#print stormcounts.head()

## Model2 -- Simply take the average total #INJURIES etc. per year per STATE per EVENT_CATEGORY; model independent of time

In [23]:
#test = dfn_trainvalid.groupby(['BEGIN_YEAR','STATE','EVENT_CATEGORY']).sum().reset_index()
#sel = test[ (test.STATE == 'missouri') & (test.BEGIN_YEAR == 2011) & (test.EVENT_CATEGORY == 'Storm')]
#sel
#test2 = test.groupby(['STATE','EVENT_CATEGORY']).median().reset_index()
#test2[ (test2.STATE == 'missouri')& (test2.EVENT_CATEGORY == 'Storm') ]

In [25]:
# Get the sum (on damage_features) per year, state, per event category
dfdam = dfn_trainvalid.groupby(['BEGIN_YEAR','STATE','EVENT_CATEGORY']).sum()[damage_features]
dfdam = dfdam.reset_index()
# Calculate the 5th percentile, median and 95th percentile of these damages
dfCIlower = dfdam.groupby(['STATE','EVENT_CATEGORY']).quantile(q=0.05)[damage_features]
dfCIlower.rename(columns=lambda x: x+"_LOWER", inplace=True)
dfCIlower = dfCIlower.reset_index()
dfCIupper = dfdam.groupby(['STATE','EVENT_CATEGORY']).quantile(q=0.95)[damage_features]
dfCIupper.rename(columns=lambda x: x+"_UPPER", inplace=True)
dfCIupper = dfCIupper.reset_index()
dfdam = dfdam.groupby(['STATE','EVENT_CATEGORY']).median()[damage_features]
dfdam = dfdam.reset_index()
merged = dfdam.merge(dfCIlower, on = ['STATE','EVENT_CATEGORY'])
merged = merged.merge(dfCIupper, on = ['STATE','EVENT_CATEGORY'])

areadf = pd.read_csv("./data/state_areas.csv", 
                    dtype = {'AREA' : float, 'STATE' : str})
areadf.STATE = areadf.STATE.str.lower()
merged = pd.merge(merged,areadf,on='STATE')
merged['DAMAGE_PROPERTY_PER_AREA'] = merged.DAMAGE_PROPERTY/merged.AREA
print merged.head(2)

     STATE EVENT_CATEGORY  INJURIES  DEATHS  DAMAGE_PROPERTY  DAMAGE_CROPS  \
0  alabama          Flood       0.0     1.0         763800.0           0.0   
1  alabama           Heat       0.5     0.0              0.0           0.0   

   INJURIES_LOWER  DEATHS_LOWER  DAMAGE_PROPERTY_LOWER  DAMAGE_CROPS_LOWER  \
0             0.0           0.0                85164.0                 0.0   
1             0.0           0.0                    0.0                 0.0   

   INJURIES_UPPER  DEATHS_UPPER  DAMAGE_PROPERTY_UPPER  DAMAGE_CROPS_UPPER  \
0            3.20           5.2           2.746282e+08           5158840.0   
1          165.65          10.1           3.929450e+04            237962.5   

      AREA  DAMAGE_PROPERTY_PER_AREA  
0  52420.0                 14.570775  
1  52420.0                  0.000000  


In [27]:
# Test print for EVENT_CATEGORY == Storm, predicted DAMAGE_PROPERTY_PER_AREA based on the simple model
dam_property_storm = merged.loc[merged.EVENT_CATEGORY == 'Storm',:]
dam_property_storm = dam_property_storm[['STATE','DAMAGE_PROPERTY_PER_AREA']]
for index, row in dam_property_storm.iterrows():
    row_string = '[\'' + str(row['STATE']) +'\',' + str(row['DAMAGE_PROPERTY_PER_AREA']) + '],'
    print row_string

['alabama',50.5387256772],
['alaska',0.0],
['american samoa',0.0],
['arizona',0.622984472322],
['arkansas',78.3952312003],
['california',9.37502672653],
['colorado',13.842584587],
['connecticut',31.9109687895],
['delaware',208.94837284],
['district of columbia',0.0],
['florida',74.3720535904],
['georgia',3.22238115271],
['hawaii',0.0],
['idaho',0.0531297490696],
['illinois',18.7163725524],
['indiana',39.6820428336],
['iowa',83.4030974002],
['kansas',37.5566980238],
['kentucky',3.83092456939],
['louisiana',15.5312344878],
['maine',22.5049462973],
['maryland',16.1188134773],
['massachusetts',187.464894827],
['michigan',17.5571271998],
['minnesota',10.0682168492],
['mississippi',47.6090188305],
['missouri',20.2480382171],
['montana',4.42981501632],
['nebraska',170.064642913],
['nevada',2.12576420794],
['new hampshire',22.0333725532],
['new jersey',121.682907257],
['new mexico',8.01028374044],
['new york',7.29456511777],
['north carolina',36.7741875546],
['north dakota',25.1562632606],
['o

In [28]:
def get_simple_predictions(merged, damage_features):
    predictions_simple = defaultdict(dict)
    for index, row in merged.iterrows():
        for feature in damage_features:
            ci_lower = feature + '_LOWER'
            ci_upper = feature + '_UPPER'
            predictions_simple[feature][(row['STATE'],row['EVENT_CATEGORY'])]=\
                (row[ci_lower],row[ci_upper])
    return predictions_simple

In [29]:
def get_simple_accuracy(dftest, predictions, damage_features):
    success = 0
    for index, row in dftest.iterrows():
        for feature in damage_features:
            pred = predictions[feature][(row['STATE'],row['EVENT_CATEGORY'])]
            if pred[0] <= row[feature] and row[feature] <= pred[1]:
                success += 1
    return 1.0*success/(len(damage_features)*len(dftest))

### Predictions for years 2013, 2014 and 2015

In [30]:
test_13 = dfn_test.loc[dfn.BEGIN_YEAR == 2013,:]
test_14 = dfn_test.loc[dfn.BEGIN_YEAR == 2014,:]
test_15 = dfn_test.loc[dfn.BEGIN_YEAR == 2015,:]

In [31]:
# Test on 2013
bystevent13 = test_13.groupby(['STATE','EVENT_CATEGORY']).sum()[damage_features]
bystevent13 = bystevent13.reset_index()
bystevent13 = bystevent13[ bystevent13.STATE != 'virgin islands']
# Test on 2014
bystevent14 = test_14.groupby(['STATE','EVENT_CATEGORY']).sum()[damage_features]
bystevent14 = bystevent14.reset_index()
bystevent14 = bystevent14[ bystevent14.STATE != 'virgin islands']
# Test on 2015
bystevent15 = test_15.groupby(['STATE','EVENT_CATEGORY']).sum()[damage_features]
bystevent15 = bystevent15.reset_index()
bystevent15 = bystevent15[ bystevent15.STATE != 'virgin islands']

In [32]:
predictions = get_simple_predictions(merged, damage_features)
print get_simple_accuracy(bystevent13, predictions, damage_features)
print get_simple_accuracy(bystevent14, predictions, damage_features)
print get_simple_accuracy(bystevent15, predictions, damage_features)

0.633254716981
0.633844339623
0.624410377358
